# Lesson 1 — How does the robot know what to do?

---

## 🎬 Watch first — then do

**Before touching anything, watch the video below.**
It shows you exactly what you will do in this lesson.

<div style="padding:56.25% 0 0 0;position:relative;">
  <iframe src="https://player.vimeo.com/video/YOUR_VIDEO_ID"
          style="position:absolute;top:0;left:0;width:100%;height:100%;"
          frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen>
  </iframe>
</div>

---

## 🚀 Now do it yourself

You just watched it. Now reproduce it exactly.

**Step 1 — Open your desktop**

Click on the cell below then press **Shift+Enter** to run it. A button will appear.

In [1]:
import os
from urllib.parse import quote
from IPython.display import HTML, display

user = os.environ.get('JUPYTERHUB_USER', 'user')
desktop_url = f'https://app.rosforge.com/user/{quote(user, safe="")}/proxy/6080/'

display(HTML(f'''
<div style="text-align:center;margin:20px 0;">
  <a href="{desktop_url}" target="_blank"
     style="background:#00D4FF;color:#000;padding:20px 60px;text-decoration:none;
            border-radius:8px;font-size:20px;font-weight:700;display:inline-block;">
    🖥️ Open Desktop
  </a>
</div>
'''))


**Step 2 — Open Terminator**

Inside the desktop: right-click anywhere on the black background → select **Terminator**

---

**Step 3 — Start the robot**

In Terminator, type:

```bash
bash ~/course_materials/scripts/start_robot.sh
```

Wait 5 seconds. RViz opens automatically. Your robot appears in 3D.

---

**Step 4 — Open a second terminal**

The first terminal is busy running the robot. You need a second one.

Inside Terminator: right-click → **Split Horizontally**

---

**Step 5 — Drive the robot**

In the new bottom terminal:

```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}, angular: {z: 0.5}}"
```

The robot moves in RViz. Watch it.

To stop the robot:

1. Press **Ctrl+C** to stop the publisher
2. Then run:

```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{}" --once
```

---

---

## 🔍 Explore — before we explain anything

The robot is running. Now explore what’s happening under the hood.

**In the second terminal, run these one by one:**

```bash
ros2 node list
```

```bash
ros2 topic list
```

```bash
ros2 topic echo /odom
```
*(Press Ctrl+C to stop)*

```bash
rqt_graph
```

Look at `rqt_graph` carefully. You can see who talks to who.

---

**This is what your rqt_graph should look like:**

![rqt_graph](images/rqt_graph_lesson1.png)

Every circle is a node. Every arrow is a topic. The arrow direction shows who publishes and who subscribes.

---

## 🧠 Challenge 1 — Before reading the explanation

You have seen the system running. Look at `rqt_graph`.

**Answer these questions using only what you observed — no googling:**

1. How many separate programs are running right now?
2. When you typed the `cmd_vel` command, where did it go? Who received it?
3. You stopped the `cmd_vel` command with Ctrl+C. Did the robot simulator stop? Why or why not?
4. What do you think `/odom` contains? What is its purpose?

Write your answers in the cell below — your guesses, not the correct answer.

*Write your answers here:*

1. 
2. 
3. 
4. 

---

## 💡 What just happened — the explanation

### Nodes

When you ran `ros2 node list`, you saw several programs running simultaneously. In ROS2, each independent program is called a **node**.

The robot simulator is a node. `robot_state_publisher` is a node. RViz is a node. The `cmd_vel` command you typed created a temporary node.

**Key insight:** Nodes never import each other. They don’t know each other exists. Yet they work together perfectly. How?

### Topics

Nodes communicate through **topics** — named channels where data flows.

- A node that **sends** data to a topic is a **publisher**
- A node that **reads** data from a topic is a **subscriber**

When you typed the `cmd_vel` command, you became a **publisher** on the `/cmd_vel` topic. The robot simulator was **subscribed** to that topic — it received your command and moved.

When you pressed Ctrl+C, the publisher stopped. But the robot simulator kept running — because it’s a completely separate program. It just received zero velocity commands and stopped moving.

### Why this matters

This architecture means:
- You can replace any node without touching the others
- Multiple nodes can publish to the same topic
- Multiple nodes can subscribe to the same topic
- Nodes can run on different computers and still communicate

This is the foundation of every robot system you will ever build.

---

---

## 🔋 Live example — you are now a subscriber

Run the cell below with **Shift+Enter**. It reads from `/battery_state` every second.

You are not writing a subscriber node — but this cell *is* a subscriber.
It connects to the topic, reads data, and displays it.

Watch the battery drain as the robot runs.

In [2]:
import subprocess, time
from IPython.display import display, HTML, clear_output

def get_battery():
    try:
        result = subprocess.run(
            ['ros2', 'topic', 'echo', '/battery_state', '--once', '--field', 'percentage'],
            capture_output=True, text=True, timeout=3
        )
        val = float(result.stdout.strip().replace('---','').strip())
        return round(val * 100, 1)
    except:
        return None

def battery_bar(pct):
    color = '#00D4FF' if pct > 50 else '#FFA500' if pct > 20 else '#FF4444'
    return f'''
<div style="font-family:monospace;background:#0a0a0a;border:1px solid #333;
            border-radius:8px;padding:20px;max-width:400px;">
  <div style="color:#aaa;font-size:13px;margin-bottom:8px;">🔋 /battery_state</div>
  <div style="background:#1a1a1a;border-radius:4px;height:24px;overflow:hidden;">
    <div style="background:{color};width:{pct}%;height:100%;"></div>
  </div>
  <div style="color:#fff;font-size:24px;font-weight:700;margin-top:8px;">{pct}%</div>
  <div style="color:#555;font-size:11px;margin-top:4px;">Updates every second — this cell is a ROS2 subscriber</div>
</div>'''

print('Monitoring battery... press the ■ Stop button to stop')
for _ in range(30):
    pct = get_battery()
    clear_output(wait=True)
    if pct is not None:
        display(HTML(battery_bar(pct)))
    else:
        display(HTML('<div style="color:#FF4444">⚠️ Robot not running — start start_robot.sh first</div>'))
    time.sleep(1)

## 🧠 Challenge 2 — Systems thinking

No commands needed. Just think.

**Scenario:** Your robot is moving and you want to monitor its battery level in real time. You also want to log its position every second to a file.

Answer these questions:

1. Should battery level use a **topic** or a **service**? Why? *(Hint: is it continuous data or a one-time request?)*
2. To log the robot position, which topic would you subscribe to?
3. If the logging node crashes, does the robot stop? Why or why not?
4. Two students are logged into the platform at the same time. Could they accidentally control each other’s robots? What mechanism in ROS2 would prevent that — and what mechanism in our platform adds another layer of isolation?

*Write your answers here:*

1. 
2. 
3. 
4. 

---

## ✅ What you learned

- A **node** is an independent program in a ROS2 system
- A **topic** is a named channel for continuous data
- A **publisher** sends data to a topic
- A **subscriber** receives data from a topic
- Nodes are decoupled — they don’t need to know about each other

**Next lesson:** You will write your own publisher and subscriber nodes from scratch.

---

### 📋 Commands reference

| Command | What it does |
|---|---|
| `ros2 node list` | Show all running nodes |
| `ros2 topic list` | Show all active topics |
| `ros2 topic echo /topic_name` | Print messages on a topic |
| `ros2 topic info /topic_name` | Show publishers and subscribers |
| `rqt_graph` | Visualize the node/topic graph |

In [3]:
import subprocess

result = subprocess.run(['which', 'ros2'], capture_output=True, text=True)
print("ros2 path:", result.stdout)

result = subprocess.run(['printenv', 'ROS_DOMAIN_ID'], capture_output=True, text=True)
print("ROS_DOMAIN_ID:", result.stdout)

ros2 path: /opt/ros/jazzy/bin/ros2

ROS_DOMAIN_ID: 226

